# W207 Spring 2019 Final Project
## Kaggle Competition: Forest Cover Prediction
**Pierce Coggins, Jake Mitchell, Debasish Mukhopadhyay, and Tim Slade**

# Table of Contents/Section Notes

- [Introduction](#introduction)
  - In which we discuss the problem and why it matters
  - [Housekeeping](#housekeeping)
    - In which we deal with basic prep and setup issues
- [About the Data](#aboutTheData)
  - EDA, charts, data cleaning
- [Feature Engineering](#featureEngineering)
  - Describe a basic model that we will use to test the usefulness of new features (LR or NB)
  - Normalization
  - Each added or removed feature
- [Models](#models)
  - Maybe choose 4 to test out?  Don't want this section to get too lengthy, and each model should be covered in some detail
- [Results](#results)
  - What went well, what went poorly
  - Final comparison of models on test data
- [Conclusion](#conclusion)
- [Annexes](#annexA)
  

<a id="introduction"></a>
# Introduction

In this report, we will attempt to predict the forest cover type (defined as the predominant type of tree cover) for a given area of land in Colorado given only cartographic variables as inputs.  This problem and dataset were initially posted as a Kaggle competition in 2015.  We have chosen to tackle this problem as it allows for many different machine learning techniques to be attempted and explored.  The report will go through the process of building a capable model from data cleaning through final testing.

The problem of understanding what type of vegetation is present in a difficult to access area is a surprisingly important one.  In this particular example the forests of Colorado are very diverse, and each type of tree cover has its own benefits and dangers.  For example many of the pine trees in Colorado are susceptible to the [mountain pine beetle](https://csfs.colostate.edu/forest-management/common-forest-insects-diseases/mountain-pine-beetle/), while the Spruce and Fir trees are relatively safe from the beetles.  Without directly going to every location in the mountains of Colorado, it is very difficult to distinguish these types of trees as they look very similar from the air.  It is relatively easy to get cartographic data for a large swath of the mountains however, and if it is possible to accurately predict the tree type from the cartographic information alone then all of the Colorado forest could be mapped by likely forest cover type. That information would be invaluable to firefighters and forest service personnel to direct their efforts where it will have the most impact.

If you would like to learn more about the problem or try for yourself, all information and data can be found from the kaggle competition:<br>[Kaggle's Forest Cover Type Prediction](https://www.kaggle.com/c/forest-cover-type-prediction).

<a id="housekeeping"></a>
## Housekeeping

### Importing Libraries, Helper Functions, and Loading Data

In [3]:
%%capture
# %matplotlib inline
# %matplotlib notebook
%matplotlib qt

# General libraries
import pandas as pd
import numpy as np
import os
import copy
import warnings
import statsmodels.api as sm
from scipy import stats
import math

# Plotting and printing libraries
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
import pprint

# Model-building libraries
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import normalize, MinMaxScaler, StandardScaler, RobustScaler, Normalizer, scale

# SK-learn libraries for learning
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

# SK-learn libraries for evaluation
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

# Run the helper functions notebook
%run w207_final_helper_functions.ipynb

The forest cover types we aim to predict are bundled with the features used to predict them. Our first step is therefore to separate them out, lest we accidentally let our models peek at the outcomes. We also want to split the dataset into _train_ and _test_ subsets; this will give us insight into how well our chosen models and parameters will perform against out-of-sample data.

The original dataset contained 15,120 observations. We will train our models on 90% of the data and hold out 10% for testing. We thus expect to have approximately 0.9 * 15,120 = 13,608 observations in our training dataset.

In [5]:
%%capture --no-stdout --no-display
full_data = pd.DataFrame.from_csv('./train.csv')
full_data.shape

# Separating out the labels
full_labels = full_data['Cover_Type']
full_features = full_data.drop('Cover_Type', axis=1)

# Setting seed so we get consistent results from our splitting
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(full_features, full_labels, test_size=0.10)

# Verifying our data shapes are as expected
print(f'''
{'':^16} | {'Observations':^12} | {'Features':^10} |
{'-'*46}
{'Training dataset':^16} | {X_train.shape[0]:^12} | {X_train.shape[1]:^10} |
{'Training labels':^16} | {y_train.shape[0]:^12} | {'--':^10} |
{'Test dataset':^16} | {X_test.shape[0]:^12} | {X_test.shape[1]:^10} |
{'Test labels':^16} | {y_test.shape[0]:^12} | {'--':^10} |
''')


                 | Observations |  Features  |
----------------------------------------------
Training dataset |    13608     |     54     |
Training labels  |    13608     |     --     |
  Test dataset   |     1512     |     54     |
  Test labels    |     1512     |     --     |



<a id="aboutTheData"></a>

# About the Data

The data comes from several wilderness areas in northern Colorado, specifically the Rawah Wilderness Area, Neota Wilderness Area, Comanche Peak Wilderness Area, and the Cache la Poudre Wilderness Area.  These are all fairly remote areas of Colorado which is why they were chosen, as there is less human influence in these places.

The features in the dataset are all cartographic measures of a 30x30m square plot of land.  We have 10 simple features. The 11th and 12th - `wilderness_area` and `soil_type` - are categorical variables which are represented as 4 and 40 binary columns respectively in our dataset. We therefore have a total of 10 + 4 + 40 = 54 features to work with.
The list below contains a short description of each feature, including where relevant its range, median, and mean. (See [Annex A](#annexA) for the associated code and further discussion of the exploratory data analysis).

- `Elevation`: _Elevation in meters_
  - **Range**: 1863 to 3849 | **Mean**: 2749.3 | **Median**: 2752


- `Aspect`: _Aspect in degrees azimuth. i.e., degrees clockwise from a line pointed at true North. So North = 0$^\circ$, East = 90$^\circ$, South = 180$^\circ$, and West = 270$^\circ$_
  - **Range**: 0 to 360 | **Mean**: 156.7 | **Median**: 126.0


- `Slope`: _Slope in degrees. 0$^\circ$ would indicate a flat plane; greater values represent steeper slopes._
  - **Range**: 0 to 52 | **Mean**: 16.5 | **Median**: 15.0 


- `Horizontal_Distance_To_Hydrology`: _Horizontal distance to nearest surface water features. Units unspecified._
  - **Range**: 0 to 1343 | **Mean**: 227.2 | **Median**: 180 


- `Vertical_Distance_To_Hydrology`: _Vertical distance to nearest surface water features. Units unspecified._
  - **Range**: -146 to 554 | **Mean**: 51.1 | **Median**: 32.0


- `Horizontal_Distance_To_Roadways`: _Horizontal distance to nearest roadway. Units unspecified._
  - **Range**: 0 to 6890 | **Mean**: 1714.0 | **Median**: 1316


- `Hillshade_9am`: _(0 to 255 index) - Hillshade index at 9am, summer solstice_
  - **Range**: 0 to 254 | **Mean**: 212.7 | **Median**: 220


- `Hillshade_Noon`: _(0 to 255 index) - Hillshade index at noon, summer solstice_
  - **Range**: 99 to 254 | **Mean**: 219.0 | **Median**: 223


- `Hillshade_3pm`: _(0 to 255 index) - Hillshade index at 3pm, summer solstice_
  - **Range**: 0 to 248 | **Mean**: 135.1 | **Median**: 138.0


- `Horizontal_Distance_To_Fire_Points`: _Horizontal distance to nearest wildfire ignition points. Units unspecified._
  - **Range**: 0 to 6993 | **Mean**: 1511.2 | **Median**: 1256 


- `Wilderness_Area`: _(4 binary columns, 0 = absence or 1 = presence) - Wilderness area designation_
  - % of cases - **Area 1**: 24% || **Area 2**: 3% || **Area 3**: 42% || **Area 4**: 31% 


- `Soil_Type`: _(40 binary columns, 0 = absence or 1 = presence) - Soil type designation_
  - The soil types descriptions can be found at the [Kaggle Competition Data Page](https://www.kaggle.com/c/forest-cover-type-prediction/data)

### Initial Exploration of the Challenge
The label indicating our data's categorization is contained in the`Cover_Type` variable, and is split up into 7 different designations. While the tree species discussed in the Colorado State Forest Service's [_Colorado's Major Tree Species_](https://csfs.colostate.edu/colorado-trees/colorados-major-tree-species/) article do not map perfectly to these categories, the article provides some insights that may prove useful in our categorization exercise.

#### <span style='color:blue'>Category 1</span>: 'Spruce/Fir'
- Species that might fit into this category include the **Blue Spruce** (which thrives at an altitude of 6700-11500 ft in sandy soils near moisture), the **Engelmann Spruce** (8000-11000 ft, moist north-facing slopes), the **Subalpine Fir** (8000-12000 ft, cold high-elevation forests), and the **White Fir** (7900-10200 ft, moist soils in valleys).

<center>Blue Spruce</center> | <center>Engelmann Spruce</center> | <center>Subalpine Fir</center> | <center>White Fir</center>
- | - | - | -
<img src="imgs/1_blue-spruce-tree.jpg" alt="BlueSpruce" style="width: 250px;"/>  | <img src="imgs/1_engelmann-spruce.jpg" alt="EngelmannSpruce" style="width: 250px;"/> | <img src="imgs/1_subalpine-fir.jpg" alt="SubalpineFir" style="width: 250px;"/> | <img src="imgs/1_white-fir-tree.jpg" alt="WhiteFir" style="width: 250px;"/>

#### <span style='color:blue'>Category 2</span>: 'Lodgepole Pine' and <span style='color:blue'>Category 3</span>: 'Ponderosa Pine'
- The **Lodgepole Pine** thrives in well-drained soils at high elevations (6000-11000 ft).
- The **Ponderosa Pine** thrives in dry, nutrient-poor soils at elevations of 6300-9500 ft. It is often found with Douglas Firs.

<center>Lodgepole Pine</center> | <center>Ponderosa Pine</center> |
- |-|
<img src="imgs/2_lodgepole-pine.jpg" alt="LodgepolePine" style="width: 250px;"/> | <img src="imgs/3_ponderosa-pine.jpg" alt="PonderosaPine" style="width: 250px;"/> |

#### <span style='color:blue'>Category 4</span>: 'Cottonwood/Willow'
- Species that might fit into this category include the **Plains Cottonwood** (which thrives at altitudes of 3500-6500 ft near sources of water), the **Narrowleaf Cottonwood** (5000-8000 ft, moist soils along streams), and the **Peachleaf Willow** (3500-7500 ft, near water sources).

<center>Plains Cottonwood</center> | <center>Narrowleaf Cottonwood</center> | <center>Peachleaf Willow</center> |
- |- |- |
<img src="imgs/4_plains-cottonwood.jpg" alt="PlainsCottonwood" style="width: 250px;"/> |<img src="imgs/4_narrowleaf-cottonwood.jpg" alt="NarrowleafCottonwood" style="width: 250px;"/> |<img src="imgs/4_peachleaf-willow.jpg" alt="PeachleafWillow" style="width: 250px;"/> |

#### <span style='color:blue'>Category 5</span>: 'Aspen' and <span style='color:blue'>Category 6</span>: 'Douglas Fir'
- The **Quaking Aspen** thrives at altitudes of 6500-11500 ft. While it can be in many soil types, it is especially found on sandy and gravelly slopes.
- The **Douglas Fir** thrives at altitudes of 6000-9500 ft in rocky soils of moist northern slopes.

<center>Quaking Aspen</center> | <center>Douglas Fir</center> |
- | - |
<img src="imgs/5_aspen.jpg" alt="QuakingAspen" style="width: 250px;"/> | <img src="imgs/6_douglas-fir.jpg" alt="DouglasFir" style="width: 250px;"/>|

#### <span style='color:blue'>Category 7</span>: 'Krummholz'
- Interestingly, _krummholz_ is not a species of tree; it is a type of tree formation (which can emerge among various tree species) that results from consistent long-term exposure to strong, cold winds. Per [Wikipedia](https://en.wikipedia.org/wiki/Krummholz), Subalpine Fir and Engelmann Spruce are often associated with Krummholz conditions (as is Lodgepole Pine, although that is more common in British Columbia).

<center>Krummholz Banner Tree</center> | <center>Krummholz White Pine</center> | <center>Krummholz Bristlecone</center> 
- |- |- |
<img src="imgs/7_krummholz-banner-tree.jpg" alt="KrummholzBannerTree (Photo credit to John Spooner - flickr.com, CC BY 2.0, https://commons.wikimedia.org/w/index.php?curid=5007578)" style="width: 250px;"/> | <img src="imgs/7_krummholz-white-pine.jpg" alt="KrummholzWhitePine (Photo credit to Walter Siegmund [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)] https://commons.wikimedia.org/wiki/File:Pinus_albicaulis_7872.JPG" style="width: 350px;"/> |  <img src="imgs/7_krummholz-windswept-bristlecone.jpg" alt="KrummholzBristlecone" style="width: 400px;"/> | 


### Where do we start?

The brief descriptions we've seen already suggest some avenues of exploration: altitude ranges and access to water seem to be of primary importance.

#### What can we learn from elevation alone?

One place to begin would be to plot out the idealized elevation ranges within which the various tree species thrive. There may be certain elevations where certain tree species would be far more prevalent than others. The graph below illustrates the ranges in which the species of trees discussed the Colorado State Forest Service's [_Colorado's Major Tree Species_](https://csfs.colostate.edu/colorado-trees/colorados-major-tree-species/) thrive, per the article.

<img src="imgs/altitudeRanges4.png" alt="ElevationRangesIdealized" style="width: 800px;"/>

It appears that lower elevations would be strongly suggestive of the `Cottonwood/Willow` `Cover_Type`, while higher elevations might be more suggestive of the `Spruce/Fir`, `Lodgepole Pine`, `Aspen`, and `Krummholz` `Cover_Type`s. The graph above is based upon idealized data from outside sources, though, and our actual dataset might tell a different story. The graphs below present the observed _elevation_ ranges and quartiles by `Cover_Type` in our data.

| <center>Elevation Ranges</center> | <center>Elevation Quartiles</center>
|-|-
|<img src="imgs/elevationRanges.png" alt="ElevationRanges" style="width: 600px;"/> |<img src="imgs/elevationQuartiles.png" alt="ElevationQuartiles" style="width: 600px;"/> |

When looking at the ranges, our dataset appears to differ from the idealized one in that the `Cottonwood/Willow` `Cover_Type` does not seem to occur at markedly lower elevations. When looking at the quartiles, though, patterns emerge that appear similar to what we would expect from the idealized presentation: `Cottonwood/Willow` tends to cluster at lower elevations, with the higher elevations dominated by `Spruce/Fir` and `Krummholz` cover types.

The separations are surprisingly clean, suggesting that `Elevation` will be a powerful feature in our models. It might be especially powerful if we could develop a method to cluster the altitudes into the interquartile ranges presented in the model above.

#### What if we bring water into the picture?
The other feature that the article suggests might be highly salient is moisture. How does the picture evolve if we add a measure of the distance to water to the mix?

The graph below is a scatterplot of the Euclidean distance (derived from the `Horizontal_Distance_To_Hydrology` and `Vertical_Distance_To_Hydrology` features) and the `Elevation`, with data points colored by the `Cover_Type`.

<img src="imgs/hydrologyAndElevationScatter.png" alt="HydrologyAndElevationScatter" style="width: 600px;"/>

The distance to hydrology appears to be informative: `Cover_Type`s 3, 4, and 6 are essentially not found when the distance to water exceeds 750. That said, it remains clear that `Elevation` is the predominant distinguishing feature.

#### What if we consider exposure to sunlight and wind?
From a layperson's perspective, the amount of sunlight to which a given plot of land is exposed would seem likely to influence the vegetation which thrives there. In our dataset, the `Hillshade` variables encode this information.

The plot below compares the 1st quartile, median, and 3rd quartiles for each measure of `Hillshade` for each category of `Cover_Type`.

<img src="imgs/hillshadeQuartiles.png" alt="HillshadeQuartiles" style="width: 600px;"/>

While the median `Hillshade` values appear to vary a little across categories in the morning and afternoon, the interquartile range is largely overlaps across categories. The overall impression is that `Hillshade` is unlikely to be determinative on its own.

Exposure to sunlight and wind would also be affected by the `Aspect`, which is essentially the compass direction (0$^\circ$ is true North, 90$^\circ$ is East, 180$^\circ$ is South, 270$^\circ$ is West) the plot is facing. While the exact nature of the interaction between these features may not be clear *a priori*, we can attempt to collapse the effect into a single feature by taking the first principal component of the morning and afternoon hillshade features with the `Aspect` feature.

The graph below plots this first principal component against `Elevation`, as we already know `Elevation` is a strongly informative feature.

<img src="imgs/hillshadeAspectPcaScatter.png" alt="hillshadeAspectPcaScatter" style="width: 600px;"/>

What patterns we see are weak at best. While the `Douglas Fir` category appears to be more prevalent at the margins of this first PC and the `Ponderosa Pine` appears to be slightly more prevalent nearer to zero, it is clear that the `Elevation` remains the dominant feature.

#### What about the 'Kitchen Sink' approach?

So far we've examined `Elevation`, `Hydrology`, `Aspect`, and `Hillshade` features on the basis of the write-ups regarding the various tree species. But what if we just took a look at all of our key features and how they relate to one another?

The graph below is a scatterplot matrix incorporating all of the raw simple features in our data, as well as the `Euclidean_Distance_To_Hydrology` feature we composed from the horizontal and vertical distances to hydrology.

<img src="imgs/scatterplotMatrixElevationAspectWaterHillshade.png" alt="scatterplotMatrixElevationAspectWaterHillshade" style="width: 800px;"/>

While `Elevation` remains the feature that seems to provide the cleanest separation between `Cover_Type`s, two additional features seem to perform pretty well at discriminating the `Lodgepole Pine`s: `Horizontal_Distance_To_Roadways` and `Horizontal_Distance_To_Fire_Points`.

### Cleaning the Data
While exploring the data (see [Annex A](#annexA)), we noted that the `Soil_Type7` and `Soil_Type15` variables are never true. Because there is no variation in this feature, it contributes nothing to any of our models.

In [6]:
# Removing uninformative features
full_features = full_features.drop(['Soil_Type7', 'Soil_Type15'], axis=1)

<a id="featureEngineering"></a>

# Feature Engineering

Feature engineering is a major part of any data science project.  Often times the most important feature in a data set is hidden behind several other features and needs to manually be pulled out.  We will also try to remove unimportant features to reduce the noise that is passed into our models.  In order to keep the engineered and base data sets separate we must first create a new copy of the data that we are free to manipulate.

### Euchlidean Distance to Hydrology

As we saw in the About the Data section, the `cover_types` can be visually broken up based on their distance to hydrology, both horizontally and vertically.  By combining the features into a single feature, we can reduce the overall number of features.

### Elevation of Hydrology

Elevation and Hydrology are very important features when it comes to predicting the cover type of an area.  By subtracting the vertical distance to hydrology from the elevation, we can find what the elevation of the hydrology itself it.  This may prove useful by providing a feature that would be able to discern an alpine lake vs a valley stream.

### Mean Distance to Feature

As we saw in the About the Data section, the distance metrics group the data pretty well for classification.  Engineering a new feature that is the mean distance to hydrology, fire points, and roadways gives a decent approximation for how remote an area is.

### Stony

This data set features 40 different types of soils.  When compared to the 7 possible labels, this number of soil types seems a bit extreme.  Different types of trees favor more rocky soils, and so combining all of the stony soil types into a single feature will allow a model to more easily pick up on that.

### Hillshade

In [7]:
full_features[['Hillshade_9am', 'Hillshade_3pm']].describe()

,Hillshade_9am,Hillshade_3pm
count,15120.000000,15120.000000
mean,212.704299,135.091997
std,30.561287,45.895189
min,0.000000,0.000000
25%,196.000000,106.000000
50%,220.000000,138.000000
75%,235.000000,167.000000
max,254.000000,248.000000


One thing to notice about the data is that the `Hillshade_9am` and `Hillshade_3pm` features are missing several values.  We choose to replace these values with the median value for those features.  This will allow the areas with missing values to be more accurately classified as they no longer have un-usable data.

In [8]:
engineered_features = pd.DataFrame.copy(full_features)
engineered_features['Euclidean_Distance_To_Hydrology'] = engineered_features.apply(lambda row: math.sqrt(row.Horizontal_Distance_To_Hydrology**2 + row.Vertical_Distance_To_Hydrology**2), axis=1)
engineered_features['Elevation_Of_Hydrology'] = engineered_features['Elevation']-engineered_features['Vertical_Distance_To_Hydrology']
engineered_features['Mean_Distance_To_Feature'] = (engineered_features['Horizontal_Distance_To_Hydrology']+engineered_features['Horizontal_Distance_To_Roadways']+engineered_features['Horizontal_Distance_To_Fire_Points'])/3
engineered_features['Stony'] = engineered_features[['Soil_Type1', 'Soil_Type2', 'Soil_Type6', 'Soil_Type9', 'Soil_Type12', 'Soil_Type18', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', ]].any(axis=1)
median_hillshade_9am = np.median(engineered_features['Hillshade_9am'])
engineered_features['Hillshade_9am'] = engineered_features.apply(lambda row: median_hillshade_9am if row.Hillshade_9am == 0 else row.Hillshade_9am, axis=1)
median_hillshade_3pm = np.median(engineered_features['Hillshade_3pm'])
engineered_features['Hillshade_3pm'] = engineered_features.apply(lambda row: median_hillshade_3pm if row.Hillshade_3pm == 0 else row.Hillshade_3pm, axis=1)

np.random.seed(0)
e_X_train, e_X_test, e_y_train, e_y_test = train_test_split(engineered_features, full_labels, test_size=0.10)

### How to Test Feature Changes

Without _a priori_ knowledge of how the interplay between soil types, topography, hydrology, etc. affects forest cover, we need a way to view the performance of new features.  As such we will use a simple Gaussian Naive Bayes model to do predictions, and quanitify the results using cross-validation.  We will be tracking performance across precision, recall, and f1-score.

#### Naïve Bayes

One reasonable place to begin might be a Naïve Bayes classifier. While it is unlikely that all of the features at our disposal are _strictly_ independent, we may be able to relax the assumption of independence enough to explore how a NB model performs.

We don't want a Bernoulli NB model: our features are not uniformly binary-valued. We also don't want a Multinomial NB model: per the documentation, it assumes integer feature counts. A Gaussian NB, on the other hand, might work well. While it assumes that the likelihoods of the features are Gaussian - and this is not necessarily strictly the case - it may be worth trying.

In [9]:
# Testing on the base data
cross_validate_model(GaussianNB(), X_train, y_train, name='Base Data GaussianNB', verbose=True)

# Testing on the engineered data
cross_validate_model(GaussianNB(), e_X_train, e_y_train, name='Base Data GaussianNB', verbose=True)

Model: Base Data GaussianNB, with 5 folds
			Spruce/Fir           | precision:  0.72 | recall:  0.50 | f1-score:  0.59 | 
			Lodgepole Pine       | precision:  0.13 | recall:  0.73 | f1-score:  0.22 | 
			Ponderosa Pine       | precision:  0.73 | recall:  0.43 | f1-score:  0.54 | 
			Cottonwood/Willow    | precision:  0.92 | recall:  0.68 | f1-score:  0.78 | 
			Aspen                | precision:  0.73 | recall:  0.59 | f1-score:  0.65 | 
			Douglas Fir          | precision:  0.07 | recall:  0.77 | f1-score:  0.12 | 
			Krummholz            | precision:  0.82 | recall:  0.86 | f1-score:  0.84 | 
			macro avg            | precision:  0.59 | recall:  0.65 | f1-score:  0.54 | 
			micro avg            | precision:  0.59 | recall:  0.59 | f1-score:  0.59 | 
			weighted avg         | precision:  0.76 | recall:  0.59 | f1-score:  0.65 | 

Model: Base Data GaussianNB, with 5 folds
			Spruce/Fir           | precision:  0.73 | recall:  0.53 | f1-score:  0.61 | 
			Lodgepole Pine       | precision

In this report we will use two metrics to determine how well a particular model performs, precision and recall.  All in all, just throwing a Gaussian Naive Bayes classifier at the data performed better than expected.  It achieved a 76% weighted precision across 5 fold cross validation.  

The engineered features do not provide as much improvement as hoped.  They resulted in 1-2% improvements across all of the metrics.  One positive however is that the improved features seem to help the poorly classified labels more than the already well classified labels.

### Failed Engineered Features

Not every feature that is engineered is a useful addition to the data set.  Randomly adding new features can add noise to the dataset without providing any new information.  We have listed the failed features below.  Some highlights include Mountain width and prominance from the Elevation and Slope features, and a few different ways to view the elevation of an area.

In [10]:
# engineered_features['Elevation_Away_From_Hydrology'] = engineered_features['Elevation']-engineered_features['Horizontal_Distance_To_Hydrology']
# engineered_features['Moutain_Width'] = engineered_features.apply(lambda row: row.Elevation/math.tan(math.radians(row.Slope+.1)), axis=1)
# engineered_features['Moutain_Prominence'] = engineered_features.apply(lambda row: row.Elevation/math.sin(math.radians(row.Slope+.1)), axis=1)
# engineered_features['Mean_Hillshade'] = engineered_features.apply(lambda row: (row.Hillshade_9am + row.Hillshade_Noon + row.Hillshade_3pm)/3, axis=1)
# engineered_features['Morning_Hillshade'] = engineered_features.apply(lambda row: (row.Hillshade_9am * row.Hillshade_Noon), axis=1)
# engineered_features['Norm_Horizontal_Distance_To_Hydrology'] = engineered_features['Horizontal_Distance_To_Hydrology']/(np.mean(engineered_features['Horizontal_Distance_To_Hydrology']))
# engineered_features['Norm_Elevation'] = engineered_features['Elevation']/(np.mean(engineered_features['Elevation']))
# engineered_features['Log_Elevation'] = engineered_features.apply(lambda row: math.log(row.Elevation), axis=1)

### Standardization of the Features

Standardization is a very important aspect of preparing data for consumption by machine learning models.  It brings all of the features into a simlilar range, meaning that the models will not end up with widely varying coefficients due to the differing scales of features.  By ensuring all features exist within a given range, we eliminate the possibility that a feature could dominate the weighting and prediction process simply by virtue of having a range that is a few orders of magnitude greater than that of another (potentially more meaningful) feature.   We will experiment with several different types of standardization to see which is the most effective. Specifically we will test Min-Max scaling, standard scaling, robust scaling and sklearn's normalizer.

#### How to Test Standardization: K Nearest Neighbors

One issue with Naive Bayes is that they are more or less invariant to feature scaling, and therefore cannot be used when testing different standardization methods.  We will use the `KNearestClassifier` with k = 3 when testing out our performance on scaled data.  We found that k = 3 consistently produces the best results.

In [11]:
# Testing on the unscaled data
cross_validate_model(KNeighborsClassifier(n_neighbors=3), e_X_train, e_y_train, name='3 Nearest Neighbors', verbose=True)

Model: 3 Nearest Neighbors, with 5 folds
			Spruce/Fir           | precision:  0.66 | recall:  0.74 | f1-score:  0.70 | 
			Lodgepole Pine       | precision:  0.58 | recall:  0.71 | f1-score:  0.64 | 
			Ponderosa Pine       | precision:  0.74 | recall:  0.83 | f1-score:  0.78 | 
			Cottonwood/Willow    | precision:  0.96 | recall:  0.90 | f1-score:  0.93 | 
			Aspen                | precision:  0.96 | recall:  0.83 | f1-score:  0.89 | 
			Douglas Fir          | precision:  0.85 | recall:  0.80 | f1-score:  0.82 | 
			Krummholz            | precision:  0.97 | recall:  0.88 | f1-score:  0.92 | 
			macro avg            | precision:  0.82 | recall:  0.81 | f1-score:  0.81 | 
			micro avg            | precision:  0.82 | recall:  0.82 | f1-score:  0.82 | 
			weighted avg         | precision:  0.84 | recall:  0.82 | f1-score:  0.82 | 



The `KNearestClassifier` performed remarkably well on the base data with 83% precision and 81% recall.  This is a remarkably good result for a very basic model.  However with such a low K, it is important to keep overfitting in mind.

#### MinMax Feature Scaling

The `MinMaxScaler` module, scales the minimum value in the set to the lower bound, and the max to the upper bound.  All points in-between are adjusted to the new scale.  This has the effect of grouping inliers together, making the data consistend, but also makes the data very narrow.

A range of [-1, 1] may be preferable to a range of [0, 1] because the `Elevation` and `Vertical_Distance_To_Hydrology` variables could legitimately include negative values. (`Elevation` does not, at least in our training data, but it could.)  

Using the range of [0,1], however, will ensure that the majority of our features make sense when scaled.  This will make features like `Slope` and `Horizontal_Distance_To_Hydrology` make more sense.  It will also keep our binary features the same.

In [12]:
mm_neg1_1_scaled_df = apply_scaler(MinMaxScaler(feature_range=(-1, 1)), e_X_train)
print("MinMaxScaler [-1,1]")
mm_neg1_1_scaled_df.head(5)

MinMaxScaler [-1,1]


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Euclidean_Distance_To_Hydrology,Elevation_Of_Hydrology,Mean_Distance_To_Feature,Stony
0,0.112790,-0.644444,-0.153846,-0.900223,-0.534286,-0.812171,0.795918,0.161290,-0.384615,-0.830688,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.898119,0.146846,-0.822992,1.0
1,-0.503525,-0.616667,-0.153846,-0.728965,-0.337143,-0.795202,0.826531,0.200000,-0.392713,-0.740312,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.703309,-0.557394,-0.747331,1.0
2,-0.325277,-0.700000,-0.769231,-0.865972,-0.577143,-0.266823,0.693878,0.651613,0.109312,-0.304733,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.867316,-0.287487,-0.246480,-1.0
3,-0.441088,0.944444,-0.346154,-0.873418,-0.540000,-0.596255,0.316327,0.419355,0.279352,-0.876162,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.872782,-0.419855,-0.730620,-1.0
4,-0.673716,0.277778,0.115385,-0.256888,-0.171429,-0.841720,0.051020,0.961290,0.716599,-0.730588,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.234510,-0.792141,-0.717623,1.0


Scaling data to the range [-1, 1] appears to result in values that are incoherent. For instance, the values for the `Slope` and `Horizontal_Distance_To_Hydrology` features are now negative, which does not make conceptual sense. 

In [13]:
mm_0_1_scaled_df = apply_scaler(MinMaxScaler(feature_range=(0, 1)), e_X_train)
print("MinMaxScaler [0,1]")
mm_0_1_scaled_df.head(5)

MinMaxScaler [0,1]


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Euclidean_Distance_To_Hydrology,Elevation_Of_Hydrology,Mean_Distance_To_Feature,Stony
0,0.556395,0.177778,0.423077,0.049888,0.232857,0.093915,0.897959,0.580645,0.307692,0.084656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.050940,0.573423,0.088504,1.0
1,0.248238,0.191667,0.423077,0.135517,0.331429,0.102399,0.913265,0.600000,0.303644,0.129844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.148345,0.221303,0.126335,1.0
2,0.337362,0.150000,0.115385,0.067014,0.211429,0.366589,0.846939,0.825806,0.554656,0.347633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.066342,0.356256,0.376760,0.0
3,0.279456,0.972222,0.326923,0.063291,0.230000,0.201872,0.658163,0.709677,0.639676,0.061919,...,0.0,0.0,0.0,0.0,0.0,0.0,0.063609,0.290072,0.134690,0.0
4,0.163142,0.638889,0.557692,0.371556,0.414286,0.079140,0.525510,0.980645,0.858300,0.134706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.382745,0.103930,0.141188,1.0


The results of the alternative scaler using the range [0, 1] seem to be a little more coherent. The primary feature whose transformation may be unexpected would be the `Vertical_Distance_To_Hydrology`, because in unscaled form it contained negative values. We should examine it under scaled conditions.

#### Standard Feature Scaling [0,1]

The `StandardScaler` scales the data to be within [0,1] by default, and also scales the data to a unit variance.  Using the `Standard Scaler` should help our scaled data stay in nice distributions.

In [14]:
standard_scaled_df = apply_scaler(StandardScaler(), e_X_train)
print("StandardScaler [0,1]")
standard_scaled_df.head(5)

StandardScaler [0,1]


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Euclidean_Distance_To_Hydrology,Elevation_Of_Hydrology,Mean_Distance_To_Feature,Stony
0,0.524048,-0.846207,0.645370,-0.764506,-0.558748,-0.807802,0.695076,-1.307127,-1.312308,-0.834120,...,-0.08205,-0.025726,-0.044588,-0.228456,-0.213266,-0.174919,-0.776211,0.610912,-1.001750,0.950237
1,-0.942188,-0.800788,0.645370,-0.219322,0.563945,-0.763844,0.793233,-1.175931,-1.334638,-0.545296,...,-0.08205,-0.025726,-0.044588,-0.228456,-0.213266,-0.174919,-0.165322,-1.032989,-0.772814,0.950237
2,-0.518129,-0.937044,-1.247065,-0.655470,-0.802812,0.604931,0.367886,0.354698,0.049816,0.846726,...,-0.08205,-0.025726,-0.044588,-0.228456,-0.213266,-0.174919,-0.679617,-0.402948,0.742659,-1.052369
3,-0.793647,1.751748,0.053984,-0.679173,-0.591290,-0.248469,-0.842715,-0.432483,0.518743,-0.979446,...,-0.08205,-0.025726,-0.044588,-0.228456,-0.213266,-0.174919,-0.696759,-0.711934,-0.722251,-1.052369
4,-1.347079,0.661697,1.473311,1.283491,1.507658,-0.884350,-1.693408,1.404272,1.724558,-0.514220,...,-0.08205,-0.025726,-0.044588,-0.228456,-0.213266,-0.174919,1.304749,-1.580957,-0.682925,0.950237


#### Robust Feature Scaling

The `RobustScaler` behaves very similarly to the `MinMaxScaler` but it uses the inter-quartile range to scale features rather than just min and max.  This should allow it to be more robust to outliers in the data.

In [15]:
r_scaled_df = apply_scaler(RobustScaler(), e_X_train)
print("RobustScaler [0,1]")
r_scaled_df.head(5)

RobustScaler [0,1]


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Euclidean_Distance_To_Hydrology,Elevation_Of_Hydrology,Mean_Distance_To_Feature,Stony
0,0.297799,-0.316327,0.583333,-0.429658,-0.206667,-0.443780,0.358974,-1.196429,-1.016667,-0.525896,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.439403,0.342742,-0.555832,0.0
1,-0.544017,-0.290816,0.583333,0.007605,0.713333,-0.405248,0.435897,-1.089286,-1.033333,-0.274104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.040931,-0.572581,-0.368834,0.0
2,-0.300550,-0.367347,-0.750000,-0.342205,-0.406667,0.794552,0.102564,0.160714,0.000000,0.939442,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.363453,-0.221774,0.869025,-1.0
3,-0.458735,1.142857,0.166667,-0.361217,-0.233333,0.046504,-0.846154,-0.482143,0.350000,-0.652590,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.376931,-0.393817,-0.327533,-1.0
4,-0.776479,0.530612,1.166667,1.212928,1.486667,-0.510879,-1.512821,1.017857,1.250000,-0.247012,...,0.0,0.0,0.0,0.0,0.0,0.0,1.196829,-0.877688,-0.295411,0.0


#### Normalizer Feature Scaling

The `Normalizer` is a very extreme form of normalization that is generally used when all of the features are in similar ranges.  What is does is scales all features to the same range, and forms a hypersphere of all the feature maxes.  This means that all features are within 1 unit length of the center via cartesian distance across all feature planes.  This is not an ideal case for this scaler, but it may be interesting to see what happens.

In [16]:
n_scaled_df = apply_scaler(Normalizer(), e_X_train)
print(n_scaled_df.shape)
n_scaled_df.head(10)

(13608, 56)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Euclidean_Distance_To_Hydrology,Elevation_Of_Hydrology,Mean_Distance_To_Feature,Stony
0,0.688584,0.014848,0.005104,0.015544,0.003944,0.148946,0.054289,0.043848,0.017864,0.137346,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.016037,0.684640,0.100612,0.000232
1,0.664805,0.019470,0.006208,0.051356,0.024267,0.197523,0.066876,0.054178,0.021445,0.256215,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.056801,0.640538,0.168365,0.000282
2,0.479038,0.010212,0.001135,0.017021,0.000378,0.473931,0.042363,0.042930,0.026098,0.459747,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.017025,0.478659,0.316900,0.000000
3,0.638199,0.092378,0.004487,0.022435,0.003959,0.364233,0.049356,0.055163,0.041966,0.114285,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.022781,0.634240,0.166984,0.000000
4,0.650569,0.068418,0.008627,0.148438,0.042836,0.160932,0.047893,0.074665,0.063361,0.280218,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.154495,0.607733,0.196529,0.000297
5,0.621685,0.000701,0.002103,0.069414,0.014023,0.305467,0.048379,0.051885,0.035525,0.291912,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.070816,0.607662,0.222264,0.000234
6,0.617939,0.042740,0.001796,0.141690,0.043279,0.384304,0.036096,0.044716,0.033223,0.184430,...,0.0,0.0,0.0,0.00018,0.0,0.0,0.148152,0.574660,0.236808,0.000180
7,0.659991,0.108395,0.004810,0.000000,0.000000,0.285419,0.059970,0.069912,0.054518,0.091398,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.659991,0.125606,0.000000
8,0.628566,0.021647,0.005682,0.016235,0.011094,0.343371,0.065752,0.053305,0.020023,0.238925,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.019663,0.617472,0.199510,0.000000
9,0.612945,0.091384,0.004154,0.052182,0.027000,0.360861,0.049326,0.054778,0.041019,0.275189,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.058753,0.585945,0.229411,0.000000


In [17]:
# Testing on the [-1,1] scaled data again for reference
cross_validate_model(KNeighborsClassifier(n_neighbors=3), mm_neg1_1_scaled_df, e_y_train, name='3 Nearest Neighbors, MinMax scaled [-1,1]')
# Testing on the [0,1] scaled data
cross_validate_model(KNeighborsClassifier(n_neighbors=3), mm_0_1_scaled_df, e_y_train, name='3 Nearest Neighbors, MinMax scaled [0,1]')
# Testing on the [0,1] scaled data
cross_validate_model(KNeighborsClassifier(n_neighbors=3), standard_scaled_df, e_y_train, name='3 Nearest Neighbors, Standard scaled [0,1]')
# Testing on the Robust scaled data
cross_validate_model(KNeighborsClassifier(n_neighbors=3), r_scaled_df, e_y_train, name='3 Nearest Neighbors, Robust scaled [0,1]')
# Testing on the Normalized data
cross_validate_model(KNeighborsClassifier(n_neighbors=3), n_scaled_df, e_y_train, name='3 Nearest Neighbors, Normalized')
# Testing on the unscaled data again for reference
cross_validate_model(KNeighborsClassifier(n_neighbors=3), e_X_train, e_y_train, name='3 Nearest Neighbors')

Model: 3 Nearest Neighbors, MinMax scaled [-1,1]
			weighted avg         | precision:  0.82 | recall:  0.81 | f1-score:  0.81 | 
Model: 3 Nearest Neighbors, MinMax scaled [0,1]
			weighted avg         | precision:  0.82 | recall:  0.81 | f1-score:  0.81 | 
Model: 3 Nearest Neighbors, Standard scaled [0,1]
			weighted avg         | precision:  0.81 | recall:  0.80 | f1-score:  0.80 | 
Model: 3 Nearest Neighbors, Robust scaled [0,1]
			weighted avg         | precision:  0.81 | recall:  0.80 | f1-score:  0.80 | 
Model: 3 Nearest Neighbors, Normalized
			weighted avg         | precision:  0.73 | recall:  0.70 | f1-score:  0.71 | 
Model: 3 Nearest Neighbors
			weighted avg         | precision:  0.84 | recall:  0.82 | f1-score:  0.82 | 


This test is to determine how much the scaled features affect the GaussianNB model, so we have included the previous basic KNearestNeighbors results for referance.  

First, it appears that the range that the features are scaled using the `MinMaxScaler` does not change the results.  This makes sense as the ratios between the features are linear to each other when using a `MinMaxScaler`.  Therefore we will use the [0,1] range as it causes the majority of our features to still make sense.

The `StandardScaler` acutally performs slightly worse than the `MinMaxScaler` when used on a GaussianNB model.  The two perform very similarly by all measures, but the `StandardScaler` is just slightly worse.

The `RobustScaler` averages out to the same performance as the others.

Not surprisingly the `Normalizer` did not improve the results on this data set.  This is likely because the features have such extreme differences in ranges and variances.  For example, the `Slope` feature goes from 0-54, while the `Horizontal_Distance_To_Fire_Points` ranges from 0-6993.

Looking at the results from the scaled data, we can see that our precision rose slightly, while our recall dropped by about 0.12.  This means that with the scaled features we get more of the labels correct, but we get fewer of the relevant results correct.  Looking at each label specifically we can see that `Cottonwood/Willow` jumps up to 1.00 precision and 0.46 recall, this is very suprising as 100% precision is very rare.  

In many of the sklearn models that we will be using, the models themselves perform some standardization before training.  Overall, manually standardizing the features has not improved performance in any way.

<a id="models"></a>

# Models

### Logistic Regression

Logistic Regression is a great way to look at data.  It can be pulled apart to find the coefficients for individual features, and when given enough data can perform remarkably well.

In [23]:
cross_validate_model(LogisticRegression(), e_X_train, e_y_train, name='Logistic Regression', verbose=True)

Model: Logistic Regression, with 5 folds
			Spruce/Fir           | precision:  0.65 | recall:  0.62 | f1-score:  0.64 | 
			Lodgepole Pine       | precision:  0.50 | recall:  0.58 | f1-score:  0.54 | 
			Ponderosa Pine       | precision:  0.54 | recall:  0.59 | f1-score:  0.56 | 
			Cottonwood/Willow    | precision:  0.88 | recall:  0.79 | f1-score:  0.83 | 
			Aspen                | precision:  0.69 | recall:  0.62 | f1-score:  0.65 | 
			Douglas Fir          | precision:  0.54 | recall:  0.56 | f1-score:  0.55 | 
			Krummholz            | precision:  0.84 | recall:  0.87 | f1-score:  0.86 | 
			macro avg            | precision:  0.66 | recall:  0.66 | f1-score:  0.66 | 
			micro avg            | precision:  0.66 | recall:  0.66 | f1-score:  0.66 | 
			weighted avg         | precision:  0.67 | recall:  0.66 | f1-score:  0.67 | 



This will give us a good baseline to compare future algorithms to.  The basic one-vs-many `LogisticRegression` classifier achieved ~0.67 across the board, meaning about 2/3 of its predictions are correct.

### K Nearest Neighbors

We discussed K Nearest Neighbors above when testing how effective different forms of scaling were.  KNN is a good fit for this type of problem because there are a large number of examples relative to the number of classes.  This means that every new data point will have many 'neighbors' to choose from.  We found that the ideal number of neighbors was three as there is enough data to prevent overfitting.

In [24]:
cross_validate_model(KNeighborsClassifier(n_neighbors=3), e_X_train, e_y_train, name='3 Nearest Neighbors', verbose=True)

Model: 3 Nearest Neighbors, with 5 folds
			Spruce/Fir           | precision:  0.66 | recall:  0.74 | f1-score:  0.70 | 
			Lodgepole Pine       | precision:  0.58 | recall:  0.71 | f1-score:  0.64 | 
			Ponderosa Pine       | precision:  0.74 | recall:  0.83 | f1-score:  0.78 | 
			Cottonwood/Willow    | precision:  0.96 | recall:  0.90 | f1-score:  0.93 | 
			Aspen                | precision:  0.96 | recall:  0.83 | f1-score:  0.89 | 
			Douglas Fir          | precision:  0.85 | recall:  0.80 | f1-score:  0.82 | 
			Krummholz            | precision:  0.97 | recall:  0.88 | f1-score:  0.92 | 
			macro avg            | precision:  0.82 | recall:  0.81 | f1-score:  0.81 | 
			micro avg            | precision:  0.82 | recall:  0.82 | f1-score:  0.82 | 
			weighted avg         | precision:  0.84 | recall:  0.82 | f1-score:  0.82 | 



What KNN does well is perform relatively well across all of the labels.  The lowest precision score it achieves is 56% on `Lodgepole Pines`.  As we saw in the About the Data section, Lodgepole pine trees live in areas that can be covered by many types of trees.  This makes classifying them especially hard with KNN as the Lodgepole covered areas often have neighbors of other cover types.

## Support Vector Machines

Support Vector Machines are a common tool in a data scientist's kit.  They generally perform well on datasets that are semi linearly separable, but they are very slow to train.  We will take a look at the efficacy of SVMs on this data set, as it may give an indication as to how close to linearly separable this data is.

In [25]:
# Basic Linear Support Vector machine 
cross_validate_model(LinearSVC(), e_X_train, e_y_train, name='linearSVC', verbose=True)

Model: linearSVC, with 5 folds
			Spruce/Fir           | precision:  0.19 | recall:  0.28 | f1-score:  0.16 | 
			Lodgepole Pine       | precision:  0.24 | recall:  0.27 | f1-score:  0.14 | 
			Ponderosa Pine       | precision:  0.35 | recall:  0.30 | f1-score:  0.22 | 
			Cottonwood/Willow    | precision:  0.54 | recall:  0.75 | f1-score:  0.52 | 
			Aspen                | precision:  0.09 | recall:  0.35 | f1-score:  0.14 | 
			Douglas Fir          | precision:  0.43 | recall:  0.20 | f1-score:  0.21 | 
			Krummholz            | precision:  0.49 | recall:  0.58 | f1-score:  0.48 | 
			macro avg            | precision:  0.33 | recall:  0.39 | f1-score:  0.27 | 
			micro avg            | precision:  0.33 | recall:  0.33 | f1-score:  0.33 | 
			weighted avg         | precision:  0.80 | recall:  0.33 | f1-score:  0.40 | 



The standard linearSVC produces very poor results.  It has extremely low recall, and predicts the `Aspen` and `Ponderosa Pine` very poorly.  We will take a look at how it does on scaled data, as SVC's generally require some standardization.  The recommended scaling is mean 0 var 1, but we will see how the existing ones do.

In [ ]:
# Basic Linear Support Vector machine 
cross_validate_model(LinearSVC(), e_X_train, e_y_train, name='linearSVC, Unscaled')
cross_validate_model(LinearSVC(), mm_neg1_1_scaled_df, e_y_train, name='linearSVC, MinMax scaled [-1,1]')
scaled_X_train = scale(e_X_train)
scaled_X_train_df = pd.DataFrame(data=scaled_X_train,    # values
                         columns=e_X_train.columns)  # 1st row as the column names
cross_validate_model(LinearSVC(), scaled_X_train_df, e_y_train, name='linearSVC, Scaled to mean=0, variance=1')

Model: linearSVC, Unscaled
			weighted avg         | precision:  0.80 | recall:  0.33 | f1-score:  0.40 | 
Model: linearSVC, MinMax scaled [-1,1]
			weighted avg         | precision:  0.68 | recall:  0.67 | f1-score:  0.67 | 


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "D:\Anaconda2\lib\site-packages\zmq\eventloop\zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "D:\Anaconda2\lib\site-packages\tornado\stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "D:\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "D:\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "D:\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "D:\Anaconda2\lib\site-packages\ipykernel\kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "D:\Anaconda2\lib\sit

It turns out that using scaled features made the performance of the `LinearSVC` even worse, and that will be the end of the Support Vector Machins.

### Random Forest Model

Random Forests can be extremely effective on data sets with a myriad of features that each contain a little information.  The base data set contains 54 features, 44 of which are binary, making building many of the trees quick and easy.  In addition, Random Forest models can be pulled apart to take a look at which features are the most effective.

In [18]:
cross_validate_model(RandomForestClassifier(), e_X_train, e_y_train, name='RandomForest', verbose=True)

Model: RandomForest, with 5 folds
			Spruce/Fir           | precision:  0.76 | recall:  0.75 | f1-score:  0.75 | 
			Lodgepole Pine       | precision:  0.66 | recall:  0.76 | f1-score:  0.70 | 
			Ponderosa Pine       | precision:  0.82 | recall:  0.80 | f1-score:  0.81 | 
			Cottonwood/Willow    | precision:  0.97 | recall:  0.93 | f1-score:  0.95 | 
			Aspen                | precision:  0.93 | recall:  0.89 | f1-score:  0.91 | 
			Douglas Fir          | precision:  0.82 | recall:  0.84 | f1-score:  0.83 | 
			Krummholz            | precision:  0.96 | recall:  0.94 | f1-score:  0.95 | 
			macro avg            | precision:  0.84 | recall:  0.84 | f1-score:  0.84 | 
			micro avg            | precision:  0.84 | recall:  0.84 | f1-score:  0.84 | 
			weighted avg         | precision:  0.85 | recall:  0.84 | f1-score:  0.85 | 



The base `RandomForestClassifier` performs quite well out of the box, achieving a respectable ~0.83 across all of our metrics.  Lets see if using standardized data makes a difference.

In [19]:
# Testing on the [0,1] scaled data
cross_validate_model(RandomForestClassifier(n_estimators = 10), e_mm_0_1_scaled_df, e_y_train, name='RandomForest, MinMax scaled [0,1]')
# Testing on the unscaled data again for reference
cross_validate_model(RandomForestClassifier(n_estimators = 10), e_X_train, e_y_train, name='RandomForest')

NameError: name 'e_mm_0_1_scaled_df' is not defined

As expected, the scaled features did not have an effect on the performance of the model.  Random Forests are made up of trees that only deal with a few features at a time, and do not care if a feature goes from [0,1] or [0,10000].  The tree's decision boundaries are set based on whatever scale that particular feature is at.

### Ensemble Model

#### Approach
In the above nalysis, we have seen some of the models were very preicssion and recall on certain types of trees. At the same time we can see that most of the models de well with Cottonwood/Willow and poorly with Longpole Pine. So, we were interested to see if we could use the strength of all different models and take a voting to maximize the accuracy of the models. We use an ensemble model for this purpose. However, we also know that if our hero model, Random Forest, is more accurate than other models, then there is a possibility that the noise level from the poorly performing model will bring down the overall accuracy of the model. So, we should write the code in such a way that we can also see inside the ensemble, how each model performed.

So far we have seen KNN, Liner SVC and Logistic Regression models besides Random Forest. We should take the opportunity to check other classifiers and see if they can beat our RF model. It is also important to model an ensemble of these various models with and optimizing different parameters to see f we can find a better alternative model. This is a costly and time consming approach. Gridsearch with multiple model ensemble with 2 - 3 parameters often take hours of processing even when we are using high end machines with many CPU cores. We, therefore, individually tested and optimized parameters for addition models such as MLP Classifier, Ada Boost, Quadratic Discriminant Analysis and Gaussian Process Classifier. Addditionally, we also varied hyper parameters for KNN, Random Forest indvidually to see the best outcome in each model. Finally, we created an ensemble model with 9 models in a "hard" and a "soft" voting model to see if we can get the best outcome.  

#### Results
Initial result showed what we suspected. The overall effect of the ensemble models were lower than the random forest. This is because the other models were adding noise to the overall. We then started process of elimination to get the best possible combnation. We found out the above RF can be beaten by ensemble model only when we use KNN and RF. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

#clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial',random_state=1)
clf1 =LinearSVC()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()
clf4 = KNeighborsClassifier()
clf5 = SVC(gamma=2, C=1)
clf6 = GaussianProcessClassifier(1.0 * RBF(1.0))
clf7 = MLPClassifier(alpha=1,activation = 'tanh',solver = 'lbfgs' )
#clf8 = AdaBoostClassifier()
clf8 = XGBClassifier(max_depth=10, learning_rate=0.3, n_estimators=200, n_jobs=4)
clf9 = QuadraticDiscriminantAnalysis()
eclf = VotingClassifier(estimators=[
#                                    ('lsvc', clf1)
#                                    , 
#                                    ('rf', clf2)
#                                    , ('gnb', clf3)
#                                   , 
                                    ('knn', clf4)
#                                   , ('svc', clf5)
#                                   , ('gpc', clf6)
#                                   , ('mlp', clf7)
                                   , ('GB', clf8)
#                                   , ('qda', clf9)
                                   ], voting='hard')

params = {
    #    'lsvc__C': [1.0, 100.0], 
    "knn__n_neighbors":[3,5,7,],"knn__weights":["uniform","distance"],
         # 'rf__n_estimators': [5, 10], "rf__criterion": ["gini", "entropy"], "rf__min_samples_split":[5,7,10]
     }

model = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
#model.fit(scaled_FX_train, Fy_train)
#testPrediction = model.predict(scaled_FX_test)
model.fit(e_X_train, e_y_train)
testPrediction = model.predict(e_X_test)
#model.fit(X_train, y_train)
#testPrediction = model.predict(X_test)
testReport = metrics.classification_report(testPrediction, e_y_test, output_dict=True)
verbose= True
#print(f'Model: {name}')
reportFields = ['precision', 'recall', 'f1-score']
fields = sorted(testReport.keys()) if verbose else ['weighted avg']
fieldLabels = [label_names[field] if field in label_names.keys() else field for field in fields]
fieldLabels[-1] = "Final"
for i in range(len(fields)):
    output = f'\t\t{fieldLabels[i]:<20} | '
    for outputField in reportFields:
        output += f'{outputField}: {np.mean(testReport[fields[i]][outputField]):>5.2f} | '
    print(output)
for clf, label in zip([
            #clf1,
            clf2, clf3, clf4
            #, clf5,clf6,clf7,clf8,clf9
            , eclf]
            , [ 
                #"LinearSVC",
                "RandomForestClassifier"
                #,"GaussianNB()"
                ,"KNeighborsClassifier"
                #,"SVC","GaussianProcessClassifier"
                #,"MLPClassifier"
                #         ,"AdaBoostClassifier","QuadraticDiscriminantAnalysis"
                ,"Ensemble"]):
     scores = cross_val_score(clf, e_X_train, e_y_train, cv=5, scoring='accuracy')
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label)) 

In [21]:
#Note: Must fit the model prior to running this function
def FeatImportance(model, dataColumns, title): 
# Calculate the feature ranking - Top 10 
    importances = model.feature_importances_ 
    indices = np.argsort(importances)[::-1] 
    print ("%s Top 10 Features\n" %title)
    for f in range(10): 
        print("%d. %s (%f)" % (f + 1, dataColumns.columns[indices[f]], importances[indices[f]])) 
    #Mean Feature Importance 
    print ("\nMean Feature Importance %.6f" %np.mean(importances))
    #Plot the feature importances of the forest 
    indices=indices[:10] 
    plt.figure() 
    plt.title(title+" Top 10 Features") 
    plt.bar(range(10), importances[indices], color="gb", align="center") 
    plt.xticks(range(10), dataColumns.columns[indices], fontsize=12, rotation=90) 
    plt.xlim([-1, 10]) 
    plt.show()



In [ ]:
# Generate Initial RF and identify most important features
initialRF = RandomForestClassifier(n_estimators = 10)
initialRF.fit(e_X_train, e_y_train)
FeatImportance(initialRF, e_X_train, "Random Forest")

By taking a look at the most important features we can see which variables are most effective in predicting the cover type.  As expected the `Elevation` of an area is the biggest predictor of the `cover_type`.  Following the elevation of an area, we see the distances to roadways, fire points, and water all having a large effect.

### Gradient Boosting

Gradient Boosted Decision Trees are a cousin to the Random Forests.  When a Random Forest is built, it builds many trees in parallel trying to maximize the information gain of each tree.  In Gradient Boosting, trees are made iteratively with each tree attempting to correct the errors of the previous one.  These tend to perform a little better than Random Forests, while maintaining their nice properties.

In [ ]:
cross_validate_model(XGBClassifier(max_depth=10, learning_rate=0.3, n_estimators=200, n_jobs=4), e_X_train, e_y_train, name=f'Gradient Boosted Decision Trees (XGBoost)')

Gradient Boosting has proven to be the most effective model so far, achieving the very high score of 0.88 across the board.  This is a pretty great result because classifying new data using a `XGBClassifier` is extremely quick despite the long time it takes to train a new model.

## Master Model Result List

In [ ]:
with warnings.catch_warnings(record=False):
    test_model(LogisticRegression(), e_X_train, e_y_train, e_X_test, e_y_test, name="LogisticRegression")
    
    test_model(GaussianNB(), e_X_train, e_y_train, e_X_test, e_y_test, name="GaussianNB")
    
    test_model(KNeighborsClassifier(n_neighbors=3), e_X_train, e_y_train, e_X_test, e_y_test, name="3 Nearest Neighbors")
    
    test_model(LinearSVC(), e_X_train, e_y_train, e_X_test, e_y_test, name='linearSVC')
    
    test_model(RandomForestClassifier(n_estimators = 10), e_X_train, e_y_train, e_X_test, e_y_test, name='RandomForest')
    
    test_model(XGBClassifier(max_depth=10, learning_rate=0.3, n_estimators=200, n_jobs=4), e_X_train, e_y_train, e_X_test, e_y_test, name=f'Gradient Boosted Decision Trees (XGBoost)')

<a id="results"></a>
    
# Results

<a id="conclusion"></a>

# Conclusion

<a id="annexes"></a>
    
# Annexes

<a id="annexA"></a>

## Annex A: Exploratory Data Analysis

This appendix contains our exploratory data analysis. This includes the code used to generate the 5-number summaries of our data reflected in the [_About the Data_](#aboutTheData) and other summaries. The most informative portions are replicated in the main body of the report

After we load the data from the source file, we examine the basic characteristics of the dataset.
  1. We expect to see all of the features discussed above represented in our column names
  1. As there is no separate dataset containing the labels for our observations, we would expect to see the 'Cover_Type' variable in our data
  1. We would expect to see a shape of (15120, 55) - the 54 features plus our label column

In [ ]:
print(f'Columns: {full_data.columns}')
print(f'Shape: {full_data.shape}')

We take a look at the first several observations to get a sense for the nature of the data.

In [ ]:
full_features.head()

We'll also want to get a high-level summary of each of our features.

In [ ]:
# Small function to give us a bird's-eye summary of the data
def five_num_summary(df, column):
    print(f'Column: {column:<35} | ' +
          f'Max value: {np.max(df[column]):>6} | ' + 
          f'Min value: {np.min(df[column]):>7.2f} | ' +
          f'Mean: {np.mean(df[column]):>7.2f} | ' +
          f'Median: {np.median(df[column]):>7.2f}')

for col_name in full_features.columns:
    five_num_summary(full_features, col_name)

### Checking Label Imbalance

It would be useful for us to understand whether we have an imbalanced dataset (i.e., one where certain labels/categories are overrepresented relative to others.) Here we'll quickly describe our training and test labels and just make sure our classes are balanced. We can do this both graphically and numerically.

*TODO: Colorize these columns by label? Do we need both train and test graphs?*

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,3), sharey=True)
bins = np.arange(8) + 0.5
ax1.hist(y_train, bins, width = 0.8)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax1.set_title('Labels')
ax2.hist(y_test, bins, width = 0.8)
ax2.set_title('Test Labels')
plt.xticks(range(8))
plt.show()

In [ ]:
print(stats.describe(full_labels))
print(stats.describe(y_test))
for i in range(0, 8):
        print(f'i = {i}: Train Ct: {(full_labels==i).sum():>5} | Test Ct: {(y_test==i).sum():>5}')

It appears that our classes are quite well-balanced in both our training data and the test data.

This is good both because we will not need to deliberately compensate for imbalances and because our model will be unable to achieve reasonable performance simply by guessing the modal category. (Doing so would give accuracy on the training set of 1741/12096 = 0.145, and then accuracy on the test set of 411/3024 = 0.136.)

*TODO: Change this to be a visual representation*

In [ ]:
plt.hist(full_features['Elevation'])

In [ ]:
N = len(full_features['Aspect'])
print(N)

In [ ]:
# Adapting code from https://stackoverflow.com/questions/22562364/circular-histogram-for-python

# Start from the center of the disc
bottom = 0

# Break the arc up into 10-degree increments
theta = range(0, 360, 10)

# Bar height is equal to the number of `Aspect` values between X and X + 10
binned_aspects = np.digitize(full_features['Aspect'], theta)
radii = [sum(binned_aspects==x) for x in range(0, len(theta))]

# Divide arc evenly between Xbands for bar width
N = 36  # b/c 10-degree arcs
width = (2 * np.pi) / N

max_height = max(radii) + 10 # Highest
ax = plt.subplot(111, polar=True)
bars = ax.bar(theta, radii, width=width, bottom=bottom)

ax.set_theta_zero_location('N') # 0 degrees North is up
ax.set_theta_direction(-1) # Go clockwise

# Use custom colors and opacity
for r, bar in zip(radii, bars):
#     bar.set_facecolor(plt.cm.jet(r / 10.))
    bar.set_facecolor('green')
    bar.set_alpha(0.8)

plt.title('Distribution of \'Aspect\' values in 10-degree increments\n',
         fontsize=16)
plt.show()

One thing of note is that the `Soil_Type7` and `Soil_Type15` are never true, so this feature tells us nothing.  These features should be removed before any modeling is done.

In [ ]:
bins = np.arange(0, 360, 10)
cut = [0, 45, 90, 135, 180, 225, 270, 315, 360]

print(bins)
pd.cut(bins, cut)

In [ ]:
full_data['Total_Hillshade'] = full_data[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']].sum(axis=1)
full_data[['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm','Total_Hillshade']].head(20)

In [ ]:
## Make 3D scatterplot to explore water, elevation, and hillshade concurrently

%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D

sparsifier = np.random.randint(0, full_features.shape[0], 5000)

sparsified = full_features.iloc[sparsifier,:]
sparse_labels = full_labels.iloc[sparsifier]
# print(f'Length of sparsified dataset\n: {sparsified}')

full_features['Euclidean_distance_to_water'] = np.sqrt(full_features['Horizontal_Distance_To_Hydrology']**2 + full_features['Vertical_Distance_To_Hydrology']**2)
dist_to_water = sparsified['Euclidean_distance_to_water']
altitude = sparsified['Elevation']
hillshade = sparsified['Hillshade_3pm']
color_dict = {1: '#A7C6ED', 2: '#BA0C2F', 3: '#651D32', 4: '#8C8985',
              5: '#212721', 6: '#002F6C', 7: '#FFC000'}
coloration = [color_dict[x] for x in sparse_labels]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(hillshade, dist_to_water, altitude, color=coloration, alpha=0.6)
# ax.title('Forest cover categorization\nby distance to water and hillshade')
ax.view_init(30, 115)
# mouse_init(rotate_btn=1, zoom_btn=3)
plt.show()

In [ ]:
pca = PCA(n_components=1)
data = sparsified[['Hillshade_3pm', 'Hillshade_9am', 'Aspect']]
X = pca.fit_transform(data)
coloration = [color_dict[x] for x in sparse_labels]
print(data.shape, X.shape, sparsified['Elevation'].shape)
plt.scatter(sparsified['Elevation'], X, color=coloration)

In [ ]:
fire_X = np.array(sparsified[['Horizontal_Distance_To_Fire_Points', 'Horizontal_Distance_To_Roadways']])
# fire_Y = np.array(sparsified['Horizontal_Distance_To_Roadways']).reshape(-1, 1)
fire_Y = sparse_labels
fire_X.shape, fire_Y.shape

regressor = LinearRegression().fit(fire_X, fire_Y)
regressor.coef_

In [ ]:
plt.scatter(sparsified['Horizontal_Distance_To_Fire_Points'], sparsified['Horizontal_Distance_To_Roadways'], color=coloration)

In [ ]:
plt.scatter(sparsified['Elevation'], sparsified['Euclidean_distance_to_water'],color=coloration)

In [ ]:
# Adapting code from https://stackoverflow.com/questions/22562364/circular-histogram-for-python
# Start from the center of the disc
bottom = 0

# Break the arc up into 10-degree increments
theta = range(0, 360, 10)

# Bar height is equal to the number of `Aspect` values between X and X + 10
binned_aspects = np.digitize(full_features['Aspect'], theta)
radii = [sum(binned_aspects==x) for x in range(0, len(theta))]

# Divide arc evenly between Xbands for bar width
N = 36  # b/c 10-degree arcs
width = (2 * np.pi) / N

max_height = max(radii) + 10 # Highest
ax = plt.subplot(111, polar=True)
bars = ax.bar(theta, radii, width=width, bottom=bottom)

ax.set_theta_zero_location('N') # 0 degrees North is up
ax.set_theta_direction(-1) # Go clockwise

# Use custom colors and opacity
for r, bar in zip(radii, bars):
#     bar.set_facecolor(plt.cm.jet(r / 10.))
    bar.set_facecolor('green')
    bar.set_alpha(0.8)

plt.title('Distribution of \'Aspect\' values in 10-degree increments\n',
         fontsize=16)
plt.show()

In [ ]:
plt.scatter(sparsified['Aspect'], sparsified['Hillshade_9am'], color=coloration)

In [ ]:
plt.scatter(sparsified['Aspect'], sparsified['Hillshade_Noon'], color=coloration)

In [ ]:
plt.scatter(sparsified['Aspect'], sparsified['Hillshade_3pm'], color=coloration)

In [ ]:
plt.scatter(sparsified['Horizontal_Distance_To_Hydrology'], sparsified['Aspect'],color=coloration)

In [ ]:
fig.savefig('imgs/3dSparsifiedElevationHydrologyHillshade.png', bbox_inches='tight')

In [ ]:
def make_scatter(X, Y, color_scheme, alpha=1, title='MissingTitle', save=None, verbose=False):
    plt.scatter(X, Y, color=color_scheme, alpha=alpha)
    plt.title(title)
    lgd = plt.legend(handles=[cat_1, cat_2, cat_3, cat_4, cat_5, cat_6, cat_7],
           bbox_to_anchor=(1, 1),
           bbox_transform=plt.gcf().transFigure)
    if save is not None:
        plt.savefig(save, bbox_extra_artists=(lgd,), bbox_inches='tight')
        if verbose:
            print(f'Figure saved to {save}')
    plt.show()

In [ ]:
Xs = ['Elevation', 'Aspect', 'Euclidean_distance_to_water',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
Ys = ['Elevation', 'Aspect', 'Euclidean_distance_to_water',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']

for x in Xs:
    for y in Ys:
        make_scatter(sparsified[x], sparsified[y], coloration, 0.7,
                     str(f'{x} vs. {y}'), str(f'imgs/scatterplot{x}_vs_{y}'))

In [ ]:
pd.plotting.scatter_matrix(sparsified[['Elevation','Aspect', 'Euclidean_distance_to_water',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']], figsize=(16, 16), color=coloration)
plt.savefig('imgs/scatterplotMatrixElevationAspectWaterHillshade.pdf', bbox_inches='tight')

In [ ]:

pca = PCA(n_components=1)
data = sparsified[['Hillshade_3pm', 'Hillshade_9am', 'Aspect']]
X = pca.fit_transform(data)
coloration = [color_dict[x] for x in sparse_labels]
print(data.shape, X.shape, sparsified['Elevation'].shape)
plt.scatter(sparsified['Elevation'], X, color=coloration)
cat_1 = mpatches.Patch(color=color_dict[1], label='Spruce/Fir')
cat_2 = mpatches.Patch(color=color_dict[2], label='Lodgepole Pine')
cat_3 = mpatches.Patch(color=color_dict[3], label='Ponderosa Pine')
cat_4 = mpatches.Patch(color=color_dict[4], label='Cottonwood/Willow')
cat_5 = mpatches.Patch(color=color_dict[5], label='Aspen')
cat_6 = mpatches.Patch(color=color_dict[6], label='Douglas Fir')
cat_7 = mpatches.Patch(color=color_dict[7], label='Krummholz')
# plt.legend(handles=[cat_1, cat_2, cat_3, cat_4, cat_5, cat_6, cat_7])
plt.legend(handles=[cat_1, cat_2, cat_3, cat_4, cat_5, cat_6, cat_7],
           bbox_to_anchor=(1, 1),
           bbox_transform=plt.gcf().transFigure)

#### Graphs - Elevations